In [176]:
import glob2
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from utils import *

In [10]:
# google search images
len(glob2.glob("/mnt/sda/dataset/google_search_results/batch1/original/*/*"))

39989

In [11]:
# copy and rename google search files
success,not_found,errors,file_map_google=copy_map_and_rename(glob2.glob("/mnt/sda/dataset/google_search_results/batch1/original/*/*"),\
     "/mnt/sda/dataset/google_insta_reddit/original")

print("{} files copied successfully!, out of {}".format(success,\
            len(glob2.glob("/mnt/sda/dataset/google_search_results/batch1/original/*/*"))))
print("Following files not copied:\n" + "\n".join(errors))
print()
print("Following files did not match pillar:\n" + "\n".join(not_found))

39989 files copied successfully!, out of 39989
Following files not copied:


Following files did not match pillar:



In [12]:
# Instagram files
print(len(glob2.glob("/mnt/sda/abdul/rum/data-scrapping/data/instagram/*/*")))

38755


In [13]:
# File names are instagram ids, getting total unique file count
len(set([file.split("/")[-1] for file in glob2.glob("/mnt/sda/abdul/rum/data-scrapping/data/instagram/*/*")]))

33546

38755-33546=5209 duplicate file names

In [14]:
# copy and rename insta files
unique_file_names = get_unique_filenames(glob2.glob("/mnt/sda/abdul/rum/data-scrapping/data/instagram/*/*"))       
success,errors,file_map_insta=copy_and_rename(unique_file_names,"/mnt/sda/dataset/google_insta_reddit/original","insta")
print("{} files copied successfully!, out of {}".format(success,len(unique_file_names)))
print("Following files not copied:\n" + "\n".join(errors))

33546 files copied successfully!, out of 33546
Following files not copied:



In [15]:
# tally
33546+39989, len(glob2.glob("/mnt/sda/dataset/google_insta_reddit/original/*"))

(73535, 73535)

In [16]:
# reddit files
print("total: ",len(glob2.glob("/mnt/sda/abdul/rum/data-scrapping/data/reddit/*/images/*")))
print("unique file names: ",len(set([file.split("/")[-1] for file in glob2.glob("/mnt/sda/abdul/rum/data-scrapping/data/reddit/*/images/*")])))
unique_file_names=get_unique_filenames(glob2.glob("/mnt/sda/abdul/rum/data-scrapping/data/reddit/*/images/*"))
success,errors,file_map_reddit=copy_and_rename(unique_file_names,"/mnt/sda/dataset/google_insta_reddit/original","reddit")
print("{} files copied successfully!, out of {}".format(success,len(unique_file_names)))
print("Following files not copied:\n" + "\n".join(errors))

total:  238037
unique file names:  237880
237880 files copied successfully!, out of 237880
Following files not copied:



In [17]:
# tally 
33546+39989+237880, len(glob2.glob("/mnt/sda/dataset/google_insta_reddit/original/*"))

(311415, 311415)

In [18]:
file_map={}
file_map.update(file_map_google)
file_map.update(file_map_insta)
file_map.update(file_map_reddit)
with open('/mnt/sda/dataset/google_insta_reddit/postprocessing/filemap.json','w') as f:
    json.dump(file_map,f)

In [31]:
# removing corrupted files
non_corrupted=return_non_corrupted_files("/mnt/sda/dataset/google_insta_reddit/original/")
non_corrupted_files=[item[0] for item in non_corrupted]
print("non corrupted: {} out of {}".format(len(non_corrupted),\
                                          len(glob2.glob("/mnt/sda/dataset/google_insta_reddit/original/*"))))

non corrupted: 282682 out of 311415


In [28]:
non_corrupted_dict={}
for item in non_corrupted:
    non_corrupted_dict[item[0]]=item[1]
with open("/mnt/sda/dataset/google_insta_reddit/postprocessing/non_corrupted.json","w") as f:
    json.dump(non_corrupted_dict,f)

In [55]:
success,errors=copy(non_corrupted_files,"/mnt/sda/dataset/google_insta_reddit/postprocessing/non_corrupted")
print("{} files copied successfully!, out of {}".format(success,len(non_corrupted_files)))
# print("Following files not copied:\n" + "\n".join(errors)) 

282682 files copied successfully!, out of 282682


In [2]:
# removing no text
# reader = easyocr.Reader(['en'], gpu=["cuda:0","cuda:1"])
files=glob2.glob("/mnt/sda/dataset/google_insta_reddit/original/*")
ocr_text,error_files,no_text_files=read_txt(files)
print("{} text extracted !, out of {}".format(len(ocr_text),len(files)))
print("file count without text: {}".format(len(no_text_files)))
print("file count extracted text: {}/{}".format(len(ocr_text),len(files)))
print("error file count: {}".format(len(error_files)))

/opt/conda/lib/python3.8/site-packages/PIL/Image.py:2896: DecompressionBombWarning: Image size (90414000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


/opt/conda/lib/python3.8/site-packages/PIL/Image.py:2896: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


/opt/conda/lib/python3.8/site-packages/PIL/Image.py:2896: DecompressionBombWarning: Image size (135473130 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


267854 text extracted !, out of 311416
file count without text: 15681
file count extracted text: 267854/311416
error file count: 27881


In [4]:
# tally
15681+27881+267854 #(1 extra file)

311416

In [6]:
with open("/mnt/sda/dataset/google_insta_reddit/postprocessing/ocr.json","w") as f:
    json.dump(str(ocr_text),f)
print("sample output ",ocr_text[list(ocr_text.keys())[0]]) 
ocr_text_filtered={}
for file,texts in ocr_text.items():
    ocr_text_filtered[file]="\n".join([text[1] for text in texts])
with open("/mnt/sda/dataset/google_insta_reddit/postprocessing/ocr_text_only.json","w") as f:
    json.dump(str(ocr_text_filtered),f) 
      
# to csv
pd.DataFrame(ocr_text_filtered.items(),columns=['file','text']).replace(regex = '\n', value = ' ').\
    to_csv('/mnt/sda/dataset/google_insta_reddit/postprocessing/ocr_text.csv',index=False)
# to json
with open("/mnt/sda/dataset/google_insta_reddit/postprocessing/ocr_text_only.json","w") as f:
    json.dump(ocr_text_filtered,f) 

sample output  [([[16, 12], [52, 12], [52, 38], [16, 38]], '2', 0.0674939737582605), ([[76, 3], [472, 3], [472, 33], [76, 33]], 'National Crime Prevention Council Singapore', 0.9937172669946763), ([[77, 33], [131, 33], [131, 53], [77, 53]], '4d*6', 0.8501265645027161), ([[14, 62], [792, 62], [792, 93], [14, 93]], "Don t be fooled by those claiming to be who they' re not; offering services they can t fulfil; or", 0.5519198740110262), ([[14, 90], [500, 90], [500, 118], [14, 118]], 'attempting to emotionally blackmail you for your money:', 0.6397081144193957), ([[12, 124], [700, 124], [700, 152], [12, 152]], 'Scammers are Still on & spree; stay cautious and protect your hard-earned money:', 0.46274645699028855), ([[14, 160], [278, 160], [278, 186], [14, 186]], 'Spot the signs; stop the crimes:', 0.7692638893263821), ([[14, 186], [354, 186], [354, 212], [14, 212]], '#spotthesigns #spreadthewordonscams', 0.9663249112814105), ([[75, 263], [345, 263], [345, 303], [75, 303]], 'MY DOG STEPPED O

In [7]:
with open('/mnt/sda/dataset/google_insta_reddit/postprocessing/filemap.json','r') as f:
    file_map=json.load(f)      
file_map_old_to_new={old:new for new,old in file_map.items()}

In [ ]:
# copy files

with_text=[]
for index,(file,_) in enumerate(non_corrupted_dict.items()):
    file_name=file.split("/")[-1]
    if file_name in ocr_text_filtered:
        with_text.append(file_name)
print("with text: {} out of {}".format(len(with_text),len(non_corrupted_dict)))

# save with text files
with open("/mnt/sda/dataset/google_insta_reddit/postprocessing/with_text.txt","w") as f:
    f.write("\n".join(with_text))
    
success=0
for index,file_name in enumerate(with_text):
    file_path="/mnt/sda/dataset/google_insta_reddit/postprocessing/non_corrupted/{}".\
                format(file_name)
    shutil.copy(file_path,\
            "/mnt/sda/dataset/google_insta_reddit/postprocessing/with_text")
    success+=1
    print("index: {}".format(index),end="\r")  
print("copied {} out of {}".format(success,\
            len(glob2.glob("/mnt/sda/dataset/google_insta_reddit/postprocessing/non_corrupted/*"))))    

with text: 267854 out of 282682


In [9]:
# Remove duplicates
duplicates=return_duplicates('/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/with_text/')
with open("/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/duplicate_output.json","w") as f:
    json.dump(duplicates,f)
    
# get unique samples
with open("/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/ocr_text_only.json","r") as f:
    ocr_text_filtered=json.load(f)
with open("/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/non_corrupted.json","r") as f:
    non_corrupted_dict=json.load(f)
    
unique_samples=get_unique_samples(duplicates,ocr_text_filtered,\
                                  {key.split("/")[-1]:size for key,size in non_corrupted_dict.items()})
print(len(unique_samples))
with open("/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/unique_samples.txt","w") as f:
    f.write("\n".join(unique_samples))
    
# class-wise duplicates
dups=[file.split("_")[0] for file in list(duplicates.keys()) if file not in unique_samples]
for item in set(dups):
    print("{} dups: {}".format(item,dups.count(item)))

231194
psychological dups: 1694
digital dups: 3190
reddit dups: 9451
civil dups: 1603
social dups: 8908
others dups: 3204
military dups: 2929
economic dups: 3106
insta dups: 2575


In [11]:
# tally
231194+1694+3190+9451+1603+8908+3204+2929+3106+2575,267854

(267854, 267854)

In [141]:
with open("/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/unique_samples.txt","r") as f:
    unique_samples=f.read().split("\n")
with open('/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/filemap.json','r') as f:
    file_map=json.load(f)
with open("/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/ocr_text_only.json","r") as f:
    ocr_text_filtered=json.load(f)
with open("/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/non_corrupted.json","r") as f:
    non_corrupted_dict=json.load(f)
non_corrupted_dict={file.split("/")[-1]:size for file,size in non_corrupted_dict.items()}    
ocr_text_filtered={file:" ".join(text.split("\n")) for file,text in ocr_text_filtered.items()}

In [142]:
# Identifying priority
top_priority_keywords=['CPF','HDB','NSF','National Service','SPF','newater','new water',\
                      'scdf','tracetogether','east coast','Heng Swee','Lee Hsing','smart nation',\
                      'sinkies','SAF','encik','chinese','indian','ndp','fresh chicken','memedef','NS']
# ocr_text_filtered={file:" ".join(text.split("\n")) for file,text in ocr_text_filtered.items()}
google_insta_text=[(file,ocr_text_filtered[file]) for file in unique_samples if "reddit_" not in file]
filtered_top_priority=[file for file,text in google_insta_text if any([token.lower() in text.lower().split(" ")\
                                                                       for token in top_priority_keywords])]
print("top priority: ",len(filtered_top_priority))

# get vocab from google searched memes
google_text=[ocr_text_filtered[file] for file in unique_samples if "insta_" not in file and "reddit_" not in file]
vect = CountVectorizer()
bow = vect.fit_transform(google_text)
total_features = len(vect.vocabulary_)
top_vect = CountVectorizer(max_features=int(total_features * 0.2))
top_bow = top_vect.fit_transform(google_text)
print("all vocab: ",len(vect.vocabulary_),"top vocab: ",len(top_vect.vocabulary_))

# remove common english words from vocab
common_eng_wrods=pd.read_csv("/mnt/sda/nirmal/rum/words_pos.csv")['word'].tolist()
useful_tokens=['singapore']+\
[word.lower() for word in list(top_vect.vocabulary_.keys()) if word not in common_eng_wrods and word.isalpha()]

google_insta_text=[(file,ocr_text_filtered[file]) for file in unique_samples if "reddit" not in file]
filtered=[(file,text) for file,text in google_insta_text if any([token.lower() in text.lower().split(" ")\
                                                                 for token in useful_tokens])]
filtered_medium_priority=[file for file,text in filtered if file not in filtered_top_priority]
print("medium priority: ",len(filtered_medium_priority))

useful_tokens=['singapore']+\
[word.lower() for word in list(vect.vocabulary_.keys()) if word not in common_eng_wrods and word.isalpha()]
filtered=[(file,text) for file,text in google_insta_text if any([token.lower() in text.lower().split(" ")\
                                                                 for token in useful_tokens])]

filtered_low_priority=[file for file,text in filtered if file not in filtered_top_priority and file not in \
                     filtered_medium_priority ]
print("low priority: ",len(filtered_low_priority))

# least priority - google,insta files not covered above
filtered_least_priority=[file for file in unique_samples if file not in [*filtered_top_priority,\
                                *filtered_medium_priority,*filtered_low_priority] and "reddit" not in file]
print("least priority: ",len(filtered_least_priority))

top priority:  1371
all vocab:  17732 top vocab:  3546
medium priority:  10756
low priority:  3662
least priority:  16497


In [143]:
# to csv

original_names=[file_map[file] for file in unique_samples]
texts=[ocr_text_filtered[file] for file in unique_samples]
text_lengths=[len(text.split(" ")) for text in texts]
platform=[file.split("_")[0] for file in unique_samples]
google_search_map=glob2.glob("/mnt/sda/abdul/rum/data-scrapping/data/google_search_images/*/*")
google_search_map=[file.replace("/mnt/sda/abdul/rum/data-scrapping/data/google_search_images/","")\
                     for file in google_search_map]
google_search_map={file.split("/")[-1]:file.split("/")[0] for file in google_search_map}
source=[google_search_map[file_map[file]] if "insta_" not in file and "reddit_" not in file else "" \
        for file in unique_samples]
platform=["instagram" if "insta_" in file else "reddit" if "reddit_" in file else "google" \
          for file in unique_samples]
keywords=[",".join([token for token in ocr_text_filtered[file].split(" ") if token.lower() in \
                    list(map(lambda x: x.lower(), [*top_priority_keywords,*useful_tokens]))])\
          if file in [*filtered_top_priority,*filtered_medium_priority,*filtered_low_priority] \
          else "" for file in unique_samples]
image_sizes=[non_corrupted_dict[file] for file in unique_samples]

 
priority={"file":unique_samples,"original file":original_names,"text":texts,"text length":text_lengths,\
         "platform":platform,"google search keywords":source,"priority keywords":keywords,"size":image_sizes}
                  
priority=pd.DataFrame(priority)                  
priority['priority']=''
for i,row in priority.iterrows():
    if(row['file'] in filtered_top_priority):
        priority.at[i,'priority'] = 'top'
    elif(row['file'] in filtered_medium_priority):
        priority.at[i,'priority'] = 'medium'
    elif(row['file'] in filtered_low_priority):
        priority.at[i,'priority'] = 'low'
    elif(row['file'] in filtered_least_priority):
        priority.at[i,'priority'] = 'least'                  
priority.to_csv('/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/priority.csv',\
                                  index=False)

batch2

In [2]:
# same file is downloaded multiple times, filtering them
files=glob2.glob("/mnt/sda/nirmal/imageye/*/*")
uniques=set([file.split("/")[-1].split("_")[0] for file in files if ("_") in file and "download" not in file])
len(uniques)   

4104

In [3]:
# from the duplicate downloads, getting the max size file for each unique file
# the web plugin downloads files with different resolution each time
filtered=[]
errored=[]
for uniq in uniques:
    duplicates=[file for file in files if uniq in file]
    sizes=[]
    for dup in duplicates:
        try:
            img = Image.open(dup)  # open the image file
            img.verify()
            sizes.append(img.size)
        except (IOError, SyntaxError) as e:
            errored.append(uniq)
            break
    if uniq not in errored:     
        max_size=max([sizes[0] for size in sizes])
        filtered.append((max_size,duplicates[sizes.index(max_size)]))  
len(errored), len(filtered)       

(146, 3958)

In [5]:
# copy to batch2
success,errors,file_map_insta=copy_and_rename([file for _,file in filtered],'/mnt/sda/dataset/google_insta_reddit/batch2/original',\
               scrape_source='insta',start_indx=33546)
print("{} files copied successfully!, out of {}".format(success,len(filtered)))
print("Following files not copied:\n" + "\n".join(errors))

3958 files copied successfully!, out of 3958
Following files not copied:



In [7]:
with open('/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/filemap.json','w') as f:
    json.dump(file_map_insta,f)

In [8]:
# removing corrupted files
non_corrupted=return_non_corrupted_files("/mnt/sda/dataset/google_insta_reddit/batch2/original/")
non_corrupted_files=[item[0] for item in non_corrupted]
print("non corrupted: {} out of {}".format(len(non_corrupted),\
                                          len(glob2.glob("/mnt/sda/dataset/google_insta_reddit/batch2/original/*"))))

non corrupted: 3958 out of 3958


In [9]:
non_corrupted_dict={}
for item in non_corrupted:
    non_corrupted_dict[item[0]]=item[1]
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/non_corrupted.json","w") as f:
    json.dump(non_corrupted_dict,f)

In [11]:
success,errors=copy(non_corrupted_files,"/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/non_corrupted")
print("{} files copied successfully!, out of {}".format(success,len(non_corrupted_files)))

3958 files copied successfully!, out of 3958


In [12]:
files=glob2.glob("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/non_corrupted/*")
ocr_text,error_files,no_text_files=read_txt(files)
print("{} text extracted !, out of {}".format(len(ocr_text),len(files)))
print("file count without text: {}".format(len(no_text_files)))
print("file count extracted text: {}/{}".format(len(ocr_text),len(files)))
print("error file count: {}".format(len(error_files)))

3797 text extracted !, out of 3958
file count without text: 161
file count extracted text: 3797/3958
error file count: 0


In [13]:
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/ocr.json","w") as f:
    json.dump(str(ocr_text),f)
print("sample output ",ocr_text[list(ocr_text.keys())[0]]) 
ocr_text_filtered={}
for file,texts in ocr_text.items():
    ocr_text_filtered[file]="\n".join([text[1] for text in texts])
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/ocr_text_only.json","w") as f:
    json.dump(str(ocr_text_filtered),f) 
      
# to csv
pd.DataFrame(ocr_text_filtered.items(),columns=['file','text']).replace(regex = '\n', value = ' ').\
    to_csv('/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/ocr_text.csv',index=False)
# to json
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/ocr_text_only.json","w") as f:
    json.dump(ocr_text_filtered,f) 

sample output  [([[13, 130], [147, 130], [147, 150], [13, 150]], 'How Malaysia enters', 0.9974318162778757), ([[157, 131], [199, 131], [199, 147], [157, 147]], 'house', 0.9999468477543129)]


In [16]:
with_text=[]
for index,(file,_) in enumerate(non_corrupted_dict.items()):
    file_name=file.split("/")[-1]
    if file_name in ocr_text_filtered:
        with_text.append(file_name)
print("with text: {} out of {}".format(len(with_text),len(non_corrupted_dict)))

# save with text files
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/with_text.txt","w") as f:
    f.write("\n".join(with_text))
    
success=0
for index,file_name in enumerate(with_text):
    file_path="/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/non_corrupted/{}".\
                format(file_name)
    shutil.copy(file_path,\
            "/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/with_text")
    success+=1
    print("index: {}".format(index),end="\r")  
print("copied {} out of {}".format(success,\
            len(glob2.glob("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/non_corrupted/*"))))  

with text: 3797 out of 3958
copied 3797 out of 3958


In [2]:
duplicates=return_duplicates('/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/with_text/')

2022-12-12 03:29:19,519: INFO Start: Calculating hashes...
100%|██████████| 3797/3797 [00:01<00:00, 3586.60it/s]
2022-12-12 03:29:20,714: INFO End: Calculating hashes!
2022-12-12 03:29:20,716: INFO Start: Evaluating hamming distances for getting duplicates
2022-12-12 03:29:20,717: INFO Start: Retrieving duplicates using Cython Brute force algorithm
100%|██████████| 3797/3797 [00:00<00:00, 8555.61it/s]
2022-12-12 03:29:21,262: INFO End: Retrieving duplicates using Cython Brute force algorithm
2022-12-12 03:29:21,262: INFO End: Evaluating hamming distances for getting duplicates
2022-12-12 03:29:21,263: INFO Start: Evaluating hamming distances for getting duplicates
2022-12-12 03:29:21,264: INFO Start: Retrieving duplicates using Cython Brute force algorithm
100%|██████████| 3797/3797 [00:00<00:00, 8553.80it/s]
2022-12-12 03:29:21,801: INFO End: Retrieving duplicates using Cython Brute force algorithm
2022-12-12 03:29:21,801: INFO End: Evaluating hamming distances for getting duplicates


Num. Duplicates: 3797 <class 'dict'>
Num. Duplicates (Removal): 3797 <class 'list'>


In [4]:
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/non_corrupted.json","r") as f:
    non_corrupted_dict=json.load(f)
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/ocr_text_only.json","r") as f:
    ocr_text_filtered=json.load(f) 
    
unique_samples=get_unique_samples(duplicates,ocr_text_filtered,\
                                  {key.split("/")[-1]:size for key,size in non_corrupted_dict.items()})    

In [5]:
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/duplicate_output.json","w") as f:
    json.dump(duplicates,f)
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/unique_samples.json","w") as f:
    json.dump(unique_samples,f)    

In [147]:
# Identifying priority
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/non_corrupted.json","r") as f:
    non_corrupted_dict=json.load(f)
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/ocr_text_only.json","r") as f:
    ocr_text_filtered=json.load(f) 
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/unique_samples.json","r") as f:
    unique_samples=json.load(f)    
    
ocr_text_filtered={file:" ".join(text.split("\n")) for file,text in ocr_text_filtered.items()}
google_insta_text=[(file,ocr_text_filtered[file]) for file in unique_samples if "reddit_" not in file]
filtered_top_priority=[file for file,text in google_insta_text if any([token.lower() in text.lower().split(" ")\
                                                                       for token in top_priority_keywords])]
print("top priority: ",len(filtered_top_priority))

useful_tokens=['singapore']+\
[word.lower() for word in list(top_vect.vocabulary_.keys()) if word not in common_eng_wrods and word.isalpha()]

google_insta_text=[(file,ocr_text_filtered[file]) for file in unique_samples if "reddit" not in file]
filtered=[(file,text) for file,text in google_insta_text if any([token.lower() in \
                                                text.lower().split(" ") for token in useful_tokens])]
filtered_medium_priority=[file for file,text in filtered if file not in filtered_top_priority]
print("medium priority: ",len(filtered_medium_priority))

useful_tokens=['singapore']+\
[word.lower() for word in list(vect.vocabulary_.keys()) if word not in common_eng_wrods and word.isalpha()]
filtered=[(file,text) for file,text in google_insta_text if any([token.lower() \
                                                    in text.lower().split(" ") for token in useful_tokens])]

filtered_low_priority=[file for file,text in filtered if file not in filtered_top_priority and file not in \
                     filtered_medium_priority ]
print("low priority: ",len(filtered_low_priority))

# least priority - google,insta files not covered above
filtered_least_priority=[file for file in unique_samples if file not in [*filtered_top_priority,\
                                *filtered_medium_priority,*filtered_low_priority] and "reddit" not in file]
print("least priority: ",len(filtered_least_priority))

top priority:  54
medium priority:  723
low priority:  462
least priority:  2554


In [151]:
# to csv
with open('/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/filemap.json','r') as f:
    file_map=json.load(f)
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/non_corrupted.json","r") as f:
    non_corrupted_dict=json.load(f)
non_corrupted_dict={file.split("/")[-1]:size for file,size in non_corrupted_dict.items()}    
    
original_names=[file_map[file] for file in unique_samples]
texts=[ocr_text_filtered[file] for file in unique_samples]
text_lengths=[len(text.split(" ")) for text in texts]
platform=[file.split("_")[0] for file in unique_samples]
source="instagram"
platform="instagram"
keywords=[",".join([token for token in ocr_text_filtered[file].split(" ") if token.lower() in \
                    list(map(lambda x: x.lower(), [*top_priority_keywords,*useful_tokens]))])\
          if file in [*filtered_top_priority,*filtered_medium_priority,*filtered_low_priority] \
              else "" for file in unique_samples]
image_sizes=[non_corrupted_dict[file] for file in unique_samples]

priority={"file":unique_samples,"original file":original_names,"text":texts,"text length":text_lengths,\
         "platform":platform,"google search keywords":source,"priority keywords":keywords,"size":image_sizes}
                  
priority=pd.DataFrame(priority)                  
priority['priority']=''
for i,row in priority.iterrows():
    if(row['file'] in filtered_top_priority):
        priority.at[i,'priority'] = 'top'
    elif(row['file'] in filtered_medium_priority):
        priority.at[i,'priority'] = 'medium'
    elif(row['file'] in filtered_low_priority):
        priority.at[i,'priority'] = 'low'
    elif(row['file'] in filtered_least_priority):
        priority.at[i,'priority'] = 'least'                  
priority.to_csv('/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/priority2.csv',\
                                  index=False)

In [154]:
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/ocr_text_only.json","r") as f:
    ocr_text_filtered=json.load(f) 
with open("/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/ocr_text_only.json","r") as f:
    ocr_text_filtered.update(json.load(f))
with open("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/non_corrupted.json","r") as f:
    non_corrupted_dict=json.load(f)
with open("/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/non_corrupted.json","r") as f:
    non_corrupted_dict.update(json.load(f)) 

ocr_text_filtered={file:" ".join(text.split("\n")) for file,text in ocr_text_filtered.items()}    
unique_samples=get_unique_samples(duplicates,ocr_text_filtered,\
                                  {key.split("/")[-1]:size for key,size in non_corrupted_dict.items()})     